In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi
import ipywidgets as widgets
from IPython.display import display, clear_output

In [9]:
# ---------- Geometry helpers ----------

def voronoi_finite_polygons_2d(vor: Voronoi, radius: float | None = None):
    """
    Reconstruct infinite Voronoi regions to finite regions (2D).
    Returns (regions, vertices), where each region is a list of vertex indices.
    """
    if vor.points.shape[1] != 2:
        raise ValueError("Only 2D Voronoi diagrams are supported.")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = 100.0

    # Map point -> list of its ridges
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    for p1, region_idx in enumerate(vor.point_region):
        vertices = vor.regions[region_idx]

        # Finite region already
        if -1 not in vertices:
            new_regions.append(vertices)
            continue

        # Reconstruct an infinite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v != -1]

        for p2, v1, v2 in ridges:
            if v1 == -1 or v2 == -1:
                # Make sure v2 is the finite vertex
                v_finite = v2 if v1 == -1 else v1

                t = vor.points[p2] - vor.points[p1]
                t /= np.linalg.norm(t)

                # Normal direction
                n = np.array([-t[1], t[0]])

                midpoint = (vor.points[p1] + vor.points[p2]) / 2.0
                direction = np.sign(np.dot(midpoint - center, n)) * n

                far_point = vor.vertices[v_finite] + direction * radius
                new_vertices.append(far_point.tolist())
                new_region.append(len(new_vertices) - 1)

        # Sort vertices counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:, 1] - c[1], vs[:, 0] - c[0])
        new_region = [v for _, v in sorted(zip(angles, new_region))]

        new_regions.append(new_region)

    return new_regions, np.asarray(new_vertices)


def _clip_poly_suth_hodgman(poly, edge_fn):
    """Sutherland-Hodgman polygon clipping against one half-plane defined by edge_fn(p) >= 0."""
    if len(poly) == 0:
        return poly

    out = []
    prev = poly[-1]
    prev_in = edge_fn(prev) >= 0

    for curr in poly:
        curr_in = edge_fn(curr) >= 0

        if curr_in:
            if not prev_in:
                out.append(_line_intersect_halfplane(prev, curr, edge_fn))
            out.append(curr)
        else:
            if prev_in:
                out.append(_line_intersect_halfplane(prev, curr, edge_fn))

        prev, prev_in = curr, curr_in

    return out


def _line_intersect_halfplane(a, b, edge_fn, iters=40):
    """
    Find intersection point of segment a->b with boundary edge_fn(p)=0 via bisection.
    Assumes a and b are on different sides (or one on boundary).
    """
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)

    fa = edge_fn(a)
    fb = edge_fn(b)

    if abs(fa) < 1e-12:
        return a
    if abs(fb) < 1e-12:
        return b

    lo, hi = a, b
    flo, fhi = fa, fb

    for _ in range(iters):
        mid = (lo + hi) / 2.0
        fmid = edge_fn(mid)
        if flo * fmid <= 0:
            hi, fhi = mid, fmid
        else:
            lo, flo = mid, fmid

    return (lo + hi) / 2.0


def clip_polygon_to_box(poly_xy: np.ndarray, xmin=0.0, xmax=10.0, ymin=0.0, ymax=10.0) -> np.ndarray:
    """
    Clip a polygon to an axis-aligned box using Sutherland-Hodgman.
    poly_xy: (N,2) array
    """
    poly = [p for p in poly_xy]

    # Define half-planes:
    # x >= xmin, x <= xmax, y >= ymin, y <= ymax
    poly = _clip_poly_suth_hodgman(poly, lambda p: p[0] - xmin)
    poly = _clip_poly_suth_hodgman(poly, lambda p: xmax - p[0])
    poly = _clip_poly_suth_hodgman(poly, lambda p: p[1] - ymin)
    poly = _clip_poly_suth_hodgman(poly, lambda p: ymax - p[1])

    if len(poly) == 0:
        return np.empty((0, 2), dtype=float)

    return np.asarray(poly, dtype=float)


# ---------- Plotting helpers ----------

def setup_10x10_axes(ax):
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.set_aspect("equal", adjustable="box")
    ax.set_xticks(range(0, 11))
    ax.set_yticks(range(0, 11))
    ax.grid(True)
    ax.set_xlabel("x")
    ax.set_ylabel("y")


def plot_question(ax, points):
    setup_10x10_axes(ax)
    ax.scatter(points[:, 0], points[:, 1], s=120, marker="o")
    for i, (x, y) in enumerate(points):
        ax.text(x + 0.15, y + 0.15, f"P{i+1}", fontsize=10)


def plot_solution(ax, points):
    setup_10x10_axes(ax)
    ax.scatter(points[:, 0], points[:, 1], s=120, marker="o")

    # Robustify: QJ "joggles" input to reduce degeneracy issues
    vor = Voronoi(points, qhull_options="Qbb Qc Qx QJ")
    regions, vertices = voronoi_finite_polygons_2d(vor, radius=100.0)

    # Draw clipped polygon boundaries (Voronoi cells clipped to the 10x10 box)
    for region in regions:
        poly = vertices[region]
        poly_clipped = clip_polygon_to_box(poly, 0, 10, 0, 10)
        if len(poly_clipped) < 2:
            continue

        closed = np.vstack([poly_clipped, poly_clipped[0]])
        ax.plot(closed[:, 0], closed[:, 1], linewidth=2)

    for i, (x, y) in enumerate(points):
        ax.text(x + 0.15, y + 0.15, f"P{i+1}", fontsize=10)


# ---------- Parsing manual input ----------

def parse_points(text: str) -> np.ndarray:
    """
    Accepts:
      - one point per line:  x,y
      - or semicolon-separated: x,y; x,y; ...
    Enforces:
      - integer coordinates only
      - no duplicate points
      - at least 2 points
    """
    raw = text.strip()
    if not raw:
        raise ValueError("No points provided.")

    parts = []
    for chunk in raw.replace(";", "\n").splitlines():
        chunk = chunk.strip()
        if not chunk:
            continue
        if "," not in chunk:
            raise ValueError(f"Bad point '{chunk}'. Use x,y format.")
        xs, ys = chunk.split(",", 1)

        xf = float(xs.strip())
        yf = float(ys.strip())
        if not xf.is_integer() or not yf.is_integer():
            raise ValueError(f"Non-integer coordinate in '{chunk}'. Use integers only (e.g., 3,7).")

        x = int(xf)
        y = int(yf)
        parts.append((x, y))

    if len(parts) < 2:
        raise ValueError("Need at least 2 points.")

    if len(set(parts)) != len(parts):
        raise ValueError("Duplicate points detected. Each point must be unique.")

    return np.asarray(parts, dtype=float)  # float dtype is fine; values are integers


# ---------- Trainer UI ----------

class VoronoiPractice:
    def __init__(self):
        self.out = widgets.Output()
        self.n_points = widgets.Dropdown(options=[4, 5,6,7,8,9,10,11,12,13], value=5, description="Random N:")
        self.mode = widgets.Dropdown(
            options=[("Normal", "normal"), ("Easy (axis + y=x edges)", "easy")],
            value="normal",
            description="Mode:",
        )
        self.btn_new = widgets.Button(description="New random question")
        self.btn_show = widgets.Button(description="Show solution")
        self.btn_hide = widgets.Button(description="Hide solution")
        self.manual = widgets.Textarea(
            value="2,2\n8,2\n5,7\n3,8\n9,8",
            description="Manual pts:",
            layout=widgets.Layout(width="420px", height="120px"),
        )
        self.btn_use_manual = widgets.Button(description="Use manual points")

        self.points = self._random_points(self.n_points.value, self.mode.value)
        self.solution_shown = False

        self.btn_new.on_click(self._on_new)
        self.btn_show.on_click(self._on_show)
        self.btn_hide.on_click(self._on_hide)
        self.btn_use_manual.on_click(self._on_manual)

        self._render()

    def _random_points(self, n, mode="normal"):
        rng = np.random.default_rng()

        # Choose integer lattice points from a "safe" interior box for nicer exam-style diagrams.
        # Change low/high to 0/10 if you want full border-inclusive sampling.
        low, high = 1, 9  # inclusive
        xs = np.arange(low, high + 1, dtype=int)

        def non_collinear(pts):
            return np.linalg.matrix_rank(pts - pts.mean(axis=0)) == 2

        def ridges_ok(pts):
            try:
                vor = Voronoi(pts, qhull_options="Qbb Qc Qx QJ")
            except Exception:
                return False

            for a, b in vor.ridge_points:
                dx = int(round(pts[b, 0] - pts[a, 0]))
                dy = int(round(pts[b, 1] - pts[a, 1]))
                # Want Voronoi ridge directions in {horizontal, vertical, slope +1}.
                # Ridge direction is perpendicular to (dx, dy), so allow site differences:
                # vertical (dx=0) -> horizontal ridge
                # horizontal (dy=0) -> vertical ridge
                # slope -1 (dx=-dy) -> slope +1 ridge
                if not (dx == 0 or dy == 0 or dx == -dy):
                    return False
            return True

        # NORMAL mode (existing behavior)
        if mode != "easy":
            grid = np.array([(x, y) for x in xs for y in xs], dtype=int)

            for _ in range(200):
                idx = rng.choice(len(grid), size=n, replace=False)
                pts = grid[idx].astype(float)
                if non_collinear(pts):
                    return pts

            return pts

        # EASY mode: keep regenerating until all Voronoi ridges are axis-aligned or slope +1
        pts = None
        for _ in range(100000):
            # Build a structured candidate pool to make "easy" ridges more likely:
            # one random row, one random column, one random anti-diagonal (x+y = s)
            row = int(rng.integers(low, high + 1))
            col = int(rng.integers(low, high + 1))
            s = int(rng.integers(low + low, high + high + 1))

            pool = set()
            for x in xs:
                pool.add((int(x), row))
            for y in xs:
                pool.add((col, int(y)))
            for x in xs:
                y = s - int(x)
                if low <= y <= high:
                    pool.add((int(x), int(y)))

            pool = np.array(list(pool), dtype=int)
            if len(pool) < n:
                continue

            idx = rng.choice(len(pool), size=n, replace=False)
            pts = pool[idx].astype(float)

            if not non_collinear(pts):
                continue
            if ridges_ok(pts):
                return pts

        # Fallback if we couldn't find an "easy" configuration quickly
        raise RuntimeError("Failed to generate easy-mode points after many attempts.")
        return self._random_points(n, "normal")

    def _render(self):
        with self.out:
            clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(5.6, 5.6))
            if self.solution_shown:
                plot_solution(ax, self.points)
                ax.set_title("Solution: Voronoi diagram (clipped to 10x10)")
            else:
                plot_question(ax, self.points)
                ax.set_title("Question: Draw the Voronoi diagram for these points")
            plt.show()

    def _on_new(self, _):
        self.points = self._random_points(self.n_points.value, self.mode.value)
        self.solution_shown = False
        self._render()

    def _on_show(self, _):
        self.solution_shown = True
        self._render()

    def _on_hide(self, _):
        self.solution_shown = False
        self._render()

    def _on_manual(self, _):
        pts = parse_points(self.manual.value)
        # optional: reject points outside the 10x10 grid
        if np.any(pts < 0) or np.any(pts > 10):
            raise ValueError("All points must lie within 0..10 for this trainer.")
        self.points = pts
        self.solution_shown = False
        self._render()

    def ui(self):
        controls = widgets.HBox([self.n_points, self.mode, self.btn_new, self.btn_show, self.btn_hide])
        manual_box = widgets.HBox([self.manual, self.btn_use_manual])
        display(widgets.VBox([controls, manual_box, self.out]))


# Create and display the trainer
trainer = VoronoiPractice()
trainer.ui()